In [200]:
import time, json, requests
import pandas as pd

# 每日疫情数据

In [201]:
#download data
def downloadDailyData():
    url = 'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5&callback=&_=%d'%int(time.time()*1000)
    data = json.loads(requests.get(url=url).json()['data'])
    return data

In [202]:
# decode area data to data frame
def getAreaDataFrame(response_data):
    area_tree = response_data['areaTree']
    df_area = pd.DataFrame(columns=['Country','Province','City','AllOrToday','confirm','suspect','dead','heal'])
    for country in area_tree:
        df_area = df_area.append({'Country':country['name'],
                   'Province':'',
                   'City':'',               
                   'AllOrToday':'All',
                   'confirm':country['total']['confirm'],
                   'suspect':country['total']['suspect'],
                   'dead':country['total']['dead'],
                   'heal':country['total']['heal']
                  },ignore_index=True)
        df_area = df_area.append({'Country':country['name'],
                   'Province':'',
                   'City':'',               
                   'AllOrToday':'Today',
                   'confirm':country['today']['confirm'],
                   'suspect':country['today']['suspect'],
                   'dead':country['today']['dead'],
                   'heal':country['today']['heal']
                  },ignore_index=True)
        if 'children' in country:
            country_tree = country['children']
            for province in country_tree:
                #print('pv:{}'.format(province['name']))
                df_area = df_area.append({'Country':country['name'],
                   'Province':province['name'],
                   'City':'',               
                   'AllOrToday':'All',
                   'confirm':province['total']['confirm'],
                   'suspect':province['total']['suspect'],
                   'dead':province['total']['dead'],
                   'heal':province['total']['heal']
                  },ignore_index=True)
                df_area = df_area.append({'Country':country['name'],
                   'Province':province['name'],
                   'City':'',               
                   'AllOrToday':'Today',
                   'confirm':province['today']['confirm'],
                   'suspect':province['today']['suspect'],
                   'dead':province['today']['dead'],
                   'heal':province['today']['heal']
                  },ignore_index=True)
                if 'children' in province:
                    province_tree = province['children']
                    for city in province_tree:
                        df_area = df_area.append({'Country':country['name'],
                           'Province':province['name'],
                           'City':city['name'],               
                           'AllOrToday':'All',
                           'confirm':city['total']['confirm'],
                           'suspect':city['total']['suspect'],
                           'dead':city['total']['dead'],
                           'heal':city['total']['heal']
                          },ignore_index=True)
                        df_area = df_area.append({'Country':country['name'],
                           'Province':province['name'],
                           'City':city['name'],               
                           'AllOrToday':'Today',
                           'confirm':city['today']['confirm'],
                           'suspect':city['today']['suspect'],
                           'dead':city['today']['dead'],
                           'heal':city['today']['heal']
                          },ignore_index=True)
    return df_area

In [203]:
def chinaSummary(data):
    data['chinaTotal']['AllOrToday'] = 'All'
    data['chinaAdd']['AllOrToday'] = 'Today'
    china_summary = pd.DataFrame.from_records([data['chinaTotal'],data['chinaAdd']])
    return china_summary

In [204]:
def getHistory(data):
    china_history_sum = pd.DataFrame.from_records(data['chinaDayList'])
    china_history_add = pd.DataFrame.from_records(data['chinaDayAddList'])
    return china_history_sum, china_history_add

In [205]:
data['lastUpdateTime']

'2020-02-03 23:58:03'

In [206]:
def saveAll(data):
    last_time = data['lastUpdateTime'].replace(':','-')
    print(last_time)
    area_df = getAreaDataFrame(data)
    area_df.to_csv('data/infections/area_{}.csv'.format(last_time),index=False)
    summary = chinaSummary(data)
    summary.to_csv('data/infections/summary_{}.csv'.format(last_time),index=False)
    history_sum,history_add = getHistory(data)
    history_sum.to_csv('data/infections/history_sum_{}.csv'.format(last_time),index=False)
    history_add.to_csv('data/infections/history_add_{}.csv'.format(last_time),index=False)

In [207]:
# Run this function daily to save data
def updateNow():
    data = downloadDailyData()
    saveAll(data)
    return data

In [208]:
data = updateNow()

2020-02-03 23-58-03


In [209]:
df_area = getAreaDataFrame(data)

# 数据查询

In [210]:
#今日概览
chinaSummary(data)

,confirm,suspect,dead,heal,AllOrToday
0,17345,21558,361,524,All
1,140,0,0,49,Today


In [211]:
#全球数据： 
df_area[(df_area['Province']=='') & (df_area['AllOrToday']=='All')]

,Country,Province,City,AllOrToday,confirm,suspect,dead,heal
0,中国,,,All,17345,0,361,524
890,日本,,,All,20,0,0,1
892,泰国,,,All,19,0,0,7
894,新加坡,,,All,18,0,0,0
896,韩国,,,All,15,0,0,0
898,澳大利亚,,,All,12,0,0,2
900,美国,,,All,11,0,0,0
902,德国,,,All,10,0,0,0
904,越南,,,All,8,0,0,0
906,马来西亚,,,All,8,0,0,0


In [223]:
#全国数据： 
df_area[(df_area['Country']=='中国')& (df_area['City']=='') & (df_area['AllOrToday']=='All')]

,Country,Province,City,AllOrToday,confirm,suspect,dead,heal
0,中国,,,All,17345,0,361,524
2,中国,湖北,,All,11177,0,350,299
40,中国,广东,,All,725,0,0,20
84,中国,浙江,,All,724,0,0,43
108,中国,河南,,All,566,0,2,16
154,中国,湖南,,All,521,0,0,22
184,中国,安徽,,All,408,0,0,14
218,中国,江西,,All,391,0,0,18
242,中国,重庆,,All,312,0,2,8
316,中国,江苏,,All,271,0,0,8


In [220]:
#各省数据： 
df_area[(df_area['Province']=='北京')& (df_area['AllOrToday']=='All')]

,Country,Province,City,AllOrToday,confirm,suspect,dead,heal
422,中国,北京,,All,212,0,1,12
424,中国,北京,海淀,All,42,0,0,0
426,中国,北京,朝阳,All,31,0,0,0
428,中国,北京,大兴,All,28,0,0,2
430,中国,北京,西城,All,26,0,0,0
432,中国,北京,外地来京,All,19,0,0,2
434,中国,北京,丰台,All,17,0,0,0
436,中国,北京,昌平,All,13,0,0,0
438,中国,北京,通州,All,13,0,0,0
440,中国,北京,石景山,All,5,0,0,0


In [214]:
#新增数据： All --> Today
df_area[(df_area['Province']=='北京') & (df_area['AllOrToday']=='Today')]

,Country,Province,City,AllOrToday,confirm,suspect,dead,heal
423,中国,北京,,Today,0,0,0,0
425,中国,北京,海淀,Today,0,0,0,0
427,中国,北京,朝阳,Today,0,0,0,0
429,中国,北京,大兴,Today,0,0,0,0
431,中国,北京,西城,Today,0,0,0,0
433,中国,北京,外地来京,Today,0,0,0,0
435,中国,北京,丰台,Today,0,0,0,0
437,中国,北京,昌平,Today,0,0,0,0
439,中国,北京,通州,Today,0,0,0,0
441,中国,北京,石景山,Today,0,0,0,0


In [215]:
#累计确诊历史
getHistory(data)[0]

,date,confirm,suspect,dead,heal
0,01.13,41,0,1,0
1,01.14,41,0,1,0
2,01.15,41,0,2,5
3,01.16,45,0,2,8
4,01.17,62,0,2,12
5,01.18,198,0,3,17
6,01.19,275,0,4,18
7,01.20,291,54,6,25
8,01.21,440,37,9,25
9,01.22,571,393,17,25


In [216]:
#新增确诊历史
getHistory(data)[1]

,date,confirm,suspect,dead,heal
0,01.20,77,27,0,0
1,01.21,149,53,3,0
2,01.22,131,257,8,0
3,01.23,259,680,8,6
4,01.24,444,1118,16,3
5,01.25,688,1309,15,11
6,01.26,769,3806,24,2
7,01.27,1771,2077,26,9
8,01.28,1459,3248,26,43
9,01.29,1737,4148,38,21


# 患者同行数据
两个来源：搜狗/人民日报

搜狗
https://sa.sogou.com/new-weball/page/sgs/epidemic/yyxw?type_page=yangshi&scene=2&clicktime=1580381964&enterid=1580381964&from=timeline&isappinstalled=0

In [217]:
sogou_trips = pd.read_json('https://hhyfeed.sogoucdn.com/js/common/epidemic-search/main.js')
sogou_trips.to_csv('data/trips/sogou_trips.csv')

人民日报 http://2019ncov.nosugartech.com/search.html?t_date=&t_no=&t_area=

In [218]:
url_trip = 'http://2019ncov.nosugartech.com/data.json?'
json_res = requests.get(url=url_trip).json()['data']
nosugar_trips = pd.DataFrame.from_records(json_res)
nosugar_trips.to_csv('data/trips/nosugar_trips.csv')